In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import Audio

# Some plotting niceties
plt.rc('figure', figsize=(10, 4))
sns.set_style('white')
sns.set_style('ticks')

import nengo
import phd

def img(array):
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
model = phd.sermo.Production()
model.trial.sequence = 'BLAA*POS1 + TII*POS2 + DAS*POS3'
freqs = [('ges-de-ccv', 'blaa', 1.2), ('ges-de-cv', 'tii', 1.8), ('ges-de-cvc', 'das', 1.1)]
t = 0.2
for gdir, ges, freq in freqs:
    path = phd.ges_path(gdir, '%s.ges' % ges.lower())
    traj = phd.vtl.parse_ges(path).trajectory(dt=model.trial.dt)
    model.add_syllable(label=ges.upper(), freq=freq, trajectory=traj)
    t += 1. / freq

In [ ]:
net = model.build()
with net:
    p_syll = nengo.Probe(net.sequence.syllable.output, synapse=0.01)
    p_gate = nengo.Probe(net.sequencer.gate, synapse=0.01)
    p_reset = nengo.Probe(net.sequencer.reset, synapse=0.01)
    p_timer = nengo.Probe(net.sequencer.timer, synapse=0.01)
    p_disinhib = [nengo.Probe(dmp.disinhibit, synapse=0.01) for dmp in net.syllables]
    p_dmps = [nengo.Probe(dmp.osc, synapse=0.01) for dmp in net.syllables]
    p_prod = nengo.Probe(net.production_info.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(t)

In [ ]:
syllables = [s[1].upper() for s in freqs]
plt.plot(sim.trange(), 
         nengo.spa.similarity(sim.data[p_syll],
                              net.sequence.vocab.create_subset(syllables)))
plt.legend(syllables, loc='best')

In [ ]:
plt.plot(sim.trange(), sim.data[p_gate])
plt.plot(sim.trange(), sim.data[p_reset])
plt.plot(sim.trange(), sim.data[p_timer])
plt.legend(['gate', 'kick', 'timer', 'timer'], loc="best")

In [ ]:
for p_dis in p_disinhib:
    plt.plot(sim.trange(), sim.data[p_dis])
plt.legend(syllables, loc="best")

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(sim.data[p_timer].T[0], sim.data[p_timer].T[1])
for i, p_dmp in enumerate(p_dmps):
    plt.subplot(2, 2, i + 2)
    plt.plot(sim.data[p_dmp].T[0], sim.data[p_dmp].T[1])

In [ ]:
img(sim.data[p_prod]);

## Running an experiment

In [ ]:
model = phd.sermo.Production()
expt = phd.experiments.ProductionExperiment(model, n_syllables=2, sequence_len=2, seed=5)
key = expt.run()
res = phd.experiments.ProductionResult.load(key)

In [ ]:
# Actual trajectory (from .ges files)
img(res.traj)
print(res.seq)
print(res.freqs)

In [ ]:
# Simulated trajectory
img(res.simtraj)
print(res.simrmse)

In [ ]:
# Trajectory from .ges file reconstructed from simulated trajectory
img(res.reconstructed)
print(res.reconstructedrmse)

In [ ]:
# Audio synthesized by VTL given .ges file
# reconstructed from simulated trajectory;
# note, no f0 gestures provided.
Audio(data=res.audio.flatten(), rate=res.fs)

In [ ]:
print("accuracy: %f" % res.accuracy)
print("timing: %f +/- %f" % (float(res.timing_mean), float(res.timing_var)))
print("cooccurring: %f, chance is %f" % (float(res.cooccur), float(res.co_chance)))

## Plotting experimental results

In [ ]:
df = phd.analysis.load_results(phd.experiments.ProductionResult, ['outneurons'])
print(df.columns)

plt.figure()
phd.plots.compare(df,
                  columns=['simrmse', 'reconstructedrmse'],
                  group_by='outneurons',
                  x_keys=['Simulated trajectory', 'Reconstructed trajectory'],
                  x_label='Syllable neurons',
                  y_label='RMSE',
                  plot_f=sns.barplot,
                  hue_order=["30", "60", "90", "120"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['accuracy'],
                  group_by='outneurons',
                  x_keys=['Accuracy'],
                  x_label='Syllable neurons',
                  y_label='Accuracy',
                  plot_f=sns.barplot,
                  hue_order=["30", "60", "90", "120"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['timing_mean', 'timing_var'],
                  group_by='outneurons',
                  x_keys=['Timing mean', 'Timing variance'],
                  x_label='Syllable neurons',
                  y_label='Timing',
                  plot_f=sns.barplot,
                  hue_order=["30", "60", "90", "120"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['cooccur'],
                  group_by='outneurons',
                  x_keys=['Cooccurrence'],
                  x_label='Coocurrence',
                  y_label='Accuracy',
                  plot_f=sns.barplot,
                  hue_order=["30", "60", "90", "120"])
plt.axhline(df["co_chance"].mean(), c='k')
plt.xlabel("")

In [ ]:
df = phd.analysis.load_results(phd.experiments.ProductionResult, ['seqneurons'])
print(df.columns)

plt.figure()
phd.plots.compare(df,
                  columns=['simrmse', 'reconstructedrmse'],
                  group_by='seqneurons',
                  x_keys=['Simulated trajectory', 'Reconstructed trajectory'],
                  x_label='Syllable neurons',
                  y_label='RMSE',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['accuracy'],
                  group_by='seqneurons',
                  x_keys=['Accuracy'],
                  x_label='Syllable neurons',
                  y_label='Accuracy',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['timing_mean', 'timing_var'],
                  group_by='seqneurons',
                  x_keys=['Timing mean', 'Timing variance'],
                  x_label='Syllable neurons',
                  y_label='Timing',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['cooccur'],
                  group_by='seqneurons',
                  x_keys=['Cooccurrence'],
                  x_label='Coocurrence',
                  y_label='Accuracy',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.axhline(df["co_chance"].mean(), c='k')
plt.xlabel("")

In [ ]:
df = phd.analysis.load_results(phd.experiments.ProductionResult, ['syllneurons'])
print(df.columns)

plt.figure()
phd.plots.compare(df,
                  columns=['simrmse', 'reconstructedrmse'],
                  group_by='syllneurons',
                  x_keys=['Simulated trajectory', 'Reconstructed trajectory'],
                  x_label='Syllable neurons',
                  y_label='RMSE',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['accuracy'],
                  group_by='syllneurons',
                  x_keys=['Accuracy'],
                  x_label='Syllable neurons',
                  y_label='Accuracy',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['timing_mean', 'timing_var'],
                  group_by='syllneurons',
                  x_keys=['Timing mean', 'Timing variance'],
                  x_label='Syllable neurons',
                  y_label='Timing',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.xlabel("")

plt.figure()
phd.plots.compare(df,
                  columns=['cooccur'],
                  group_by='syllneurons',
                  x_keys=['Cooccurrence'],
                  x_label='Coocurrence',
                  y_label='Accuracy',
                  plot_f=sns.barplot,
                  hue_order=["60", "120", "180", "240"])
plt.axhline(df["co_chance"].mean(), c='k')
plt.xlabel("")